In [127]:
import numpy as np

import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import gridspec

from scipy.interpolate import InterpolatedUnivariateSpline

from matplotlib import figure

import ipywidgets as widgets
from ipywidgets import interact, interactive
from ipywidgets import HBox, VBox
from ipywidgets import GridspecLayout
from ipywidgets import Button, Layout, jslink, IntText, IntSlider, Text

from operator import itemgetter

def create_expanded_button(description, button_style):
    return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto'))



In [68]:
SNeff_path = 'MDet_yrs_vs_SNeff.csv'
SNeff_file = open(SNeff_path, "r")

MDet_years1 = []
SNeffs1 = []

lines = SNeff_file.readlines()
for line in lines[6:36]:
    vals = line.split(',')
    MDet_years1.append(float(vals[2]))
    SNeffs1.append(float(vals[3]))
    

MDet_to_SNeff = InterpolatedUnivariateSpline(MDet_years1, SNeffs1)

In [69]:
Freeze_path = 'Freeze_outs.csv'
Freeze_file = open(Freeze_path, "r")

SNeffs2 = []
spin0 = []
spin05 = []
spin1 = []

lines = Freeze_file.readlines()
for line in lines[10:]:
    vals = line.split(',')
    SNeffs2.append(float(vals[0]))
    
    spin0.append(float(vals[1]))
    spin05.append(float(vals[2]) if len(vals[2]) > 0 else np.nan)
    spin1.append(float(vals[3]) if len(vals[3]) > 0 else np.nan)

SNeff_to_spin0 = InterpolatedUnivariateSpline(SNeffs2, spin0)
SNeff_to_spin05 = InterpolatedUnivariateSpline(SNeffs2[20:], spin05[20:])
SNeff_to_spin1 = InterpolatedUnivariateSpline(SNeffs2[27:], spin1[27:])

In [148]:

def make_plot(FIRST_CONFIG,
    
              SO_Used, 
              SO_Start,
              
              AdvSO_Used, 
              AdvSO_Start, 
              AdvSO_Duration,
              
              CHLAT1_Used, 
              CHLAT1_Start, 
              CHLAT1_Duration,
              
              CHALT2_Used, 
              CHLAT2_Start, 
              CHLAT2_Duration,
              
              SECOND_CONFIG,
              
              SO_Used_2, 
              SO_Start_2,
              
              AdvSO_Used_2, 
              AdvSO_Start_2, 
              AdvSO_Duration_2,
              
              CHLAT1_Used_2, 
              CHLAT1_Start_2, 
              CHLAT1_Duration_2,
              
              CHALT2_Used_2, 
              CHLAT2_Start_2, 
              CHLAT2_Duration_2,
             
              Update):
    
    Detectors    = [0.031, 0.062  , 0.135    , 0.135    ] 
    Efficiencies = [0.9  , 0.9    , 1.0      , 1.0      ]
    Names        = ["SO" , "AdvSO", "CHLAT 1", "CHLAT 2"]
    
    StartYears   = [SO_Start, AdvSO_Start,  CHLAT1_Start,  CHLAT2_Start]
    Durations    = [AdvSO_Start-SO_Start,  AdvSO_Duration,  CHLAT1_Duration,  CHLAT2_Duration]
    Used         = [SO_Used,  AdvSO_Used,  CHLAT1_Used,  CHALT2_Used]
    
    used_map = map(int, Used)
    used_int = list(used_map)
    st_years = np.multiply(StartYears, used_int)
    st_years = st_years[st_years != 0]
    FirstYear = np.min(st_years)
    LastYear = np.max(np.multiply(np.add(StartYears,Durations), used_int))

    increment = 1/52

    years = np.arange(FirstYear, LastYear, increment)
    det_times = []
    total_det_time = 0

    for date in years:
        for i in range(len(Detectors)):
            if Used[i]:
                if date >= StartYears[i] and date <= StartYears[i] + Durations[i]:
                    total_det_time += Detectors[i]*Efficiencies[i]*increment

        det_times.append(total_det_time)
    
    These_SNeffs = MDet_to_SNeff(det_times)

    These_spin0s = SNeff_to_spin0(These_SNeffs)
    These_spin05s = SNeff_to_spin05(These_SNeffs)
    These_spin1s = SNeff_to_spin1(These_SNeffs)
    


    #===============================================================================================
    
    StartYears_2   = [SO_Start_2, AdvSO_Start_2,  CHLAT1_Start_2,  CHLAT2_Start_2]
    Durations_2    = [AdvSO_Start_2-SO_Start_2,  AdvSO_Duration_2,  CHLAT1_Duration_2,  CHLAT2_Duration_2]
    Used_2         = [SO_Used_2,  AdvSO_Used_2,  CHLAT1_Used_2,  CHALT2_Used_2]

    used_map_2 = map(int, Used_2)
    used_int_2 = list(used_map_2)
    st_years_2 = np.multiply(StartYears_2, used_int_2)
    st_years_2 = st_years_2[st_years_2 != 0]
    FirstYear_2 = np.min(st_years_2)
    LastYear_2 = np.max(np.multiply(np.add(StartYears_2,Durations_2), used_int_2))

    years_2 = np.arange(FirstYear_2, LastYear_2, increment)
    det_times_2 = []
    total_det_time_2 = 0

    for date_2 in years_2:
        for i in range(len(Detectors)):
            if Used_2[i]:
                if date_2 >= StartYears_2[i] and date_2 <= StartYears_2[i] + Durations_2[i]:
                    total_det_time_2 += Detectors[i]*Efficiencies[i]*increment

        det_times_2.append(total_det_time_2)

    These_SNeffs_2 = MDet_to_SNeff(det_times_2)

    These_spin0s_2 = SNeff_to_spin0(These_SNeffs_2)
    These_spin05s_2 = SNeff_to_spin05(These_SNeffs_2)
    These_spin1s_2 = SNeff_to_spin1(These_SNeffs_2)


    
    #===============================================================================================


    %matplotlib inline

    fig = plt.figure(figsize=(15, 10), dpi=100, constrained_layout=False)
    #fig.suptitle('Time-series Comparason', fontsize=24)
    gs1 = fig.add_gridspec(nrows = 1, ncols = 1, wspace=0.001) 

    ax0 = plt.subplot(gs1[0, 0])
    
    peak = max(These_spin1s_2[-1], These_spin1s[-1])
    ax0.plot((2022, 2050), (peak, peak), scaley = False, c = "#bfbfbf", linestyle= (0, (1,5)) )
    
    if(FIRST_CONFIG):
        ax0.plot(years[:], These_spin0s[:], label='Spin 0', color='blue', linestyle='dotted')
        ax0.plot(years[:], These_spin05s[:], label='Spin 1/2', color='blue', linestyle='dashed')
        ax0.plot(years[:], These_spin1s[:], label='Spin 1', color='blue')
    
    if(SECOND_CONFIG):
        ax0.plot(years_2[:], These_spin0s_2[:], label='Spin 0', color='red', linestyle='dotted')
        ax0.plot(years_2[:], These_spin05s_2[:], label='Spin 1/2', color='red', linestyle='dashed')
        ax0.plot(years_2[:], These_spin1s_2[:], label='Spin 1', color='red')

    #for i in range(len(StartYears)):
        #if(Used[i]):
            #ax0.plot((StartYears[i], StartYears[i]), (100, 10000), scaley = False, c = "#878787")
            #ax0.text(StartYears[i] - 0.25, 12000, Names[i], rotation = 90, fontsize = 23, c = "#878787")
            
    peak = max(These_spin1s_2[-1], These_spin1s[-1])

    ax0.set_yscale('log')

    ax0.set_xlabel('Year', fontsize = 30)
    ax0.set_ylabel('Freeze Out Temp [MeV]', fontsize = 30)
    ax0.tick_params(axis='x', labelsize=23)
    ax0.tick_params(axis='y', labelsize=23)
    ax0.set_xlim(2022,2050)
    ax0.set_ylim(100,300000)

    ax0.legend()

    # fig.tight_layout()
    #fig.savefig("Test-Timeline-Graph.png")
    plt.show()
    
    
    
    
w = interactive(make_plot, 
              FIRST_CONFIG      = widgets.ToggleButton(value=True, description='1ST CONFIG (Blue)'),
              SO_Used           = widgets.ToggleButton(value=True, description='Simons Observatory'), 
              SO_Start          = widgets.IntSlider(value=2024, min=2024, max=2028, description='Start Year:'),

              AdvSO_Used        = widgets.ToggleButton(value=True, description='Adv. Simons') , 
              AdvSO_Start       = widgets.IntSlider(value=2029, min=2029, max=2035, description='Start Year:'), 
              AdvSO_Duration    = widgets.IntSlider(value=7, min=5, max=10, description='Duration:'),

              CHLAT1_Used       = widgets.ToggleButton(value=True, description='CHLAT #1') , 
              CHLAT1_Start      = widgets.IntSlider(value=2030, min=2030, max=2040, description='Start Year:'), 
              CHLAT1_Duration   = widgets.IntSlider(value=10, min=5, max=15, description='Duration:'),

              CHALT2_Used       = widgets.ToggleButton(value=False, description='CHLAT #2'), 
              CHLAT2_Start      = widgets.IntSlider(value=2030, min=2030, max=2040, description='Start Year:'), 
              CHLAT2_Duration   = widgets.IntSlider(value=10, min=5, max=15, description='Duration:'),

              SECOND_CONFIG     = widgets.ToggleButton(value=True, description='2ND CONFIG (Red)'),

              SO_Used_2         = widgets.ToggleButton(value=False, description='Simons Observatory'), 
              SO_Start_2        = widgets.IntSlider(value=2024, min=2024, max=2028, description='Start Year:'),

              AdvSO_Used_2      = widgets.ToggleButton(value=False, description='Adv. Simons') , 
              AdvSO_Start_2     = widgets.IntSlider(value=2029, min=2029, max=2035, description='Start Year:'), 
              AdvSO_Duration_2  = widgets.IntSlider(value=7, min=5, max=10, description='Duration:'),

              CHLAT1_Used_2     = widgets.ToggleButton(value=True, description='CHLAT #1') , 
              CHLAT1_Start_2    = widgets.IntSlider(value=2030, min=2030, max=2040, description='Start Year:'), 
              CHLAT1_Duration_2 = widgets.IntSlider(value=10, min=5, max=15, description='Duration:'),

              CHALT2_Used_2     = widgets.ToggleButton(value=True, description='CHLAT #2'), 
              CHLAT2_Start_2    = widgets.IntSlider(value=2030, min=2030, max=2040, description='Start Year:'), 
              CHLAT2_Duration_2 = widgets.IntSlider(value=10, min=5, max=15, description='Duration:'),
                
              Update            = widgets.ToggleButton(value=False, description='Run') )

print(type(w))
w.children

# This doesn't display the plot until one of the widgets has been used
#display(HBox([VBox(list(itemgetter(0,-1,-2)(w.children))), VBox(w.children[1:-2])]))
display(HBox([VBox(w.children[0:12]), VBox(w.children[12:24]), VBox(list(itemgetter(-1, -2)(w.children)))]))

<class 'ipywidgets.widgets.interaction.interactive'>
